# Introduction
Martyn has tried to model the growth of the model when grown on different sugars. We saw growth as expected for all sugars except arbutin, rhamnose, and salicin. We know experimentally the strain should grow on these sugars. So here I'll look into why the model doesn't grow on these sugars and try to fix it.

In [39]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction

In [65]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

## Rhamnose
In the rhamnose metabolism, the problem that prevents flux through the pathway is that in reaction RMPA L-Lactaldehyde is formed. This is currently a dead end and so this needs to be fixed to allow metabolism of rhamnose. I'll look into how this metabolite should be converted and linked to general metabolism. 

Nice paper talking about rhamnose degradation in various bacteria: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3870299/pdf/fmicb-04-00407.pdf

Here they show and confirm that B. subtilis has a bifunctional RhaEW protein, which can catalyze the L-rhamnulose-phosphate aldolase reaction as wel as be a lactaldehyde dehydrogenase. This protein is encoded by the yuxG gene in B. subtilis. Running a tBLASTn with the protein sequence, we find a significant hit in our strain. 

So the likely presence of this enzyme would mean that the lactaldehyde can be converted into lactate. Therefore I will add this reaction into the model.


In [67]:
#add reaction
model.add_reaction(Reaction(id='LALDDH', name = 'Llactaldehyde dehydrogenase'))

In [68]:
model.reactions.LALDDH.annotation['sbo'] = 'SBO:0000176'
model.reactions.LALDDH.annotation['rhea'] = '14280'
model.reactions.LALDDH.annotation['kegg.reaction'] = 'R01446'
model.reactions.LALDDH.annotation['ec-code'] = '1.2.1.22'

In [75]:
model.reactions.LALDDH.bounds = (0,1000)

In [70]:
model.reactions.LALDDH.add_metabolites({
    model.metabolites.lald__L_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.lac__L_c:1,
    model.metabolites.nadh_c:1,
    model.metabolites.h_c:2
})

In [71]:
model.reactions.LALDDH.gene_reaction_rule='yuxG'

Now the problem that our strain can't grow exclusively on rhamnose is fixed. we can save & commit

In [ ]:
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

# Salicin (salcn)
Our organism should grow on salicin, but current our model doesn't. I'll investigate what the problem is here and try to fix it in the correct way so our model can grow on this sugar. 

The salicin is taken up by PTS and can then the salicin-6-phosphate can be converted into g6p and 2hymeph_c. The 2hymeph_c then reacts with udpg to give salicin again. The problem here is that to be able to reconvert the 2hymeph, youi need the glucose that is formed when the 2hymeph is formed, so in the end there is no sugar avaiable for growth. This is ofcourse not correct and so will be investigated and fixed.

the 2hymeph is a phenol. Verious geobacillus species have been shown to be able to degrade phenols (see https://doi.org/10.1016/bs.aambs.2015.03.001 and doi:10.1016/j.jprot.2011.10.035 and http://production.datastore.cvt.dk/filestore?oid=5490af28bde7fcd94f4f839b&targetid=539ef0440370e0c1720065d8). 

Thermophilic bacilli have been shown to generally use the meta-pathway for cleaving the aromatic ring. This consists of hydroxylation of the aromatic to catechol, followed by ring cleavage by the catechol-2,3-dioxygenase and then a final oxidation or hydrolysis. So it would be most expected that this pathway is present in our strain too, if it can grow on salicin. I will go through each step of the pathway and ensure the enzymes are present before adding them. 
This pathway has been shown to be active on various substituted phenols and so i would expect that it works for the substiruted phenols we have in our strain too. 

The first enzyme of the proposed pathway, phenol hydrozylase (PheA) has been confirmed in G. thermodenitrificans. Also both subunits of the enzyme are annotated in our strain. This reaction catalyzes the conversion of the phenol into the correspnoding catechol by adding a -OH group.

The next step in the degradation is the catechol 2,3 dioxygenase. This enzyme is also annotated in our strain. Finally, our strain has a hydrolase annotated, as well as the following downstream steps that are involved in converting catechol into the final acetyl-Coa. 

All this taken together points to the likelihood that this pathway is responsible for the metabolism of the phenols that are produced when the salicin (or other sugars) are metabolized. Especially considering the evidence that this pathway works on a wide rande of substituted phenols. I will therefore make sure this is implemented into our model correctly.


In [13]:
model.metabolites.get_by_id('2hymeph_c').name = 'Salicyl alcohol'

In [8]:
model.reactions.SALCN6PH.annotation

{'sbo': 'SBO:0000176',
 'ec-code': '3.2.1.86',
 'kegg.reaction': 'R05134',
 'metanetx.reaction': 'MNXR109345',
 'rhea': '33127'}

In [15]:
model.metabolites.get_by_id('2hymeph_c').annotation

{'sbo': 'SBO:0000247',
 'chebi': 'CHEBI:16464',
 'kegg.compound': 'C02323',
 'metanetx.chemical': 'MNXM1855'}

## Arbutin
First I'll dig into arbutin metabolism.

Uptake into the cell is possible, via the pts system. The only following steps is via AB6PGH into G6P. This reaction produces benzenediol (hqn_c). This is a dead end metabolite and so thats why this pathway can never carry flux and our model can't grow on arbutin. 



In [15]:
#check that the model can then grow
with model:
    model.reactions.EX_glc__D_e.bounds = (0,0)
    model.reactions.EX_arbt_e.bounds = (-10,0)
    print (model.optimize().objective_value)

-0.008875098013070872


C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\util\solver.py:416: UserWarning:

solver status is 'infeasible'



In [21]:
model.reactions.AB6PGH

Reaction identifier,AB6PGH
Name,Arbutin 6-phosphate glucohydrolase
Memory address,0x028762c46bc8
Stoichiometry,arbt6p_c + h2o_c + h_c --> g6p_c + hqn_c Arbutin 6-phosphate + Water + H+ --> Glucose 6-phosphate + p-Benzenediol
GPR,RTMO03636 or RTMO02650
Lower bound,0.0
Upper bound,1000.0


In [22]:
model.reactions.AB6PGH.annotation

{'sbo': 'SBO:0000176',
 'ec-code': '3.2.1.86',
 'kegg.reaction': 'R05133',
 'metanetx.reaction': 'MNXR109344',
 'rhea': '28510'}